## 步驟7—資料探勘（data mining）- 統計檢定

In [2]:
pip install pingouin

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Users\howar\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.

  Created wheel for pingouin: filename=pingouin-0.3.10-py3-none-any.whl size=203396 sha256=14f9984896261edd4a441337113b2b4907b58f1bc00ccc2ad47b70bd8c9e4a0a
  Stored in directory: c:\users\howar\appdata\local\pip\cache\wheels\b8\92\fa\0257fec4f00e04a8801d3d5c37b2492f969a231137e376bd1f
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=fddc64feca18c39395f9ffecab63ef06f5e7438e43458908a3246672ee92857f
  Stored in directory: c:\users\howar\appdata\local\pip\cache\wheels\d6\64\cd\32819b511a488e4993f2fab909a95330289c3f4e0f6ef4676d
Successfully built pingouin littleutils


In [5]:
pip install scikit_posthocs

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'

You should consider upgrading via the 'C:\Users\howar\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.

  Created wheel for scikit-posthocs: filename=scikit_posthocs-0.6.7-py3-none-any.whl size=37886 sha256=9a6df41e026e5c0f834182c038942b46160816193ecd858410086e1212313bf3
  Stored in directory: c:\users\howar\appdata\local\pip\cache\wheels\b8\21\e6\f39794d4a6ee3d3cc5146ca80b5cd949452ad4a8fde9f6b9fc
Successfully built scikit-posthocs


In [6]:
import pandas as pd
import numpy as np
from scipy import stats
import statistics
from pingouin import pairwise_tukey
import scikit_posthocs as sp
from scipy.stats import t
from pingouin import kruskal
from scipy.stats import kruskal as stkr
from math import sqrt

In [7]:
lgb=pd.read_csv("lgb.csv")
rnn=pd.read_csv("rnn.csv")
elasticnet=pd.read_csv("elasticnet.csv")

In [8]:
rnn['larger'] = np.where( rnn['predictions'] > rnn['actual_values'], 1,0)

In [9]:
rnn['larger'].value_counts()

1    152956
0    143378
Name: larger, dtype: int64

In [10]:
152956/143378

1.066802438309922

In [11]:
lgb_test_abs=lgb["error_square"]
rnn_test_abs=rnn["error_square"]
elasticnet_test_abs=elasticnet["error_square"]
# 單純相減
lgb_test=lgb["error"]
rnn_test=rnn["error"]
elasticnet_test=elasticnet["error"]

In [12]:
# 檢驗常態性
print(stats.shapiro(lgb_test_abs))
print(stats.shapiro(rnn_test_abs))
print(stats.shapiro(elasticnet_test_abs))

#
# print(stats.shapiro(lgb_test))
# print(stats.shapiro(rnn_test))
# print(stats.shapiro(elasticnet_test))

ShapiroResult(statistic=0.850741982460022, pvalue=0.0)
ShapiroResult(statistic=0.8422949314117432, pvalue=0.0)
ShapiroResult(statistic=0.885456919670105, pvalue=0.0)
C:\Users\howar\AppData\Local\Programs\Python\Python37\lib\site-packages\scipy\stats\morestats.py:1681: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [13]:
import matplotlib.pyplot as plt
import seaborn as sns

## 簡單看一下單純相減的分布，看起來真的蠻常態的，雖然都沒過...

In [14]:
'''
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
sns.distplot(rnn_test , fit=stats.norm)
#
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
sns.distplot(elasticnet_test , fit=stats.norm)
#
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
sns.distplot(lgb_test , fit=stats.norm)
'''

'\nplt.figure(figsize=(12,4))\nplt.subplot(1,2,1)\nsns.distplot(rnn_test , fit=stats.norm)\n#\nplt.figure(figsize=(12,4))\nplt.subplot(1,2,1)\nsns.distplot(elasticnet_test , fit=stats.norm)\n#\nplt.figure(figsize=(12,4))\nplt.subplot(1,2,1)\nsns.distplot(lgb_test , fit=stats.norm)\n'

In [15]:
#同質
stats.levene(lgb_test_abs, rnn_test_abs, elasticnet_test_abs)

LeveneResult(statistic=27195.133220990345, pvalue=0.0)

In [16]:
#單純相減
stats.levene(lgb_test, rnn_test, elasticnet_test)

LeveneResult(statistic=43386.4313120569, pvalue=0.0)

## Anova(僅參考就好)

In [17]:
# 取絕對值的結果
stats.f_oneway(lgb_test_abs, rnn_test_abs, elasticnet_test_abs)

F_onewayResult(statistic=46162.19208493998, pvalue=0.0)

In [18]:
# 單純相減的結果
stats.f_oneway(lgb_test, rnn_test, elasticnet_test)

F_onewayResult(statistic=0.6820902060365664, pvalue=0.5055594290081289)

## 無母數kruskal

In [19]:
# 建立需要使用的dataframe
data_abs=pd.DataFrame({"rnn":rnn_test_abs,"lgb":lgb_test_abs,"elasticnet":elasticnet_test_abs})
data_abs = data_abs.melt(var_name='groups', value_name='values')

In [20]:
data_abs

,groups,values
0,rnn,0.097955
1,rnn,0.413533
2,rnn,0.006258
3,rnn,0.152499
4,rnn,0.122633
...,...,...
888997,elasticnet,2.106814
888998,elasticnet,0.272732
888999,elasticnet,0.272732
889000,elasticnet,0.147569


In [21]:
# 檢定多個獨立母體之中位數是否相等
# m=2時和wilcoxon等價
# 結論:拒絕三者中位數相等
kruskal(data=data_abs, dv='values', between='groups')

,Source,ddof1,H,p-unc
Kruskal,groups,2,71901.853172,0.0


In [22]:
# double check
stkr(lgb_test_abs, rnn_test_abs, elasticnet_test_abs)

KruskalResult(statistic=71901.8531719661, pvalue=0.0)

In [23]:
# kruskal posthoc
sp.posthoc_dunn(data_abs,"values","groups","bonferroni")

,elasticnet,lgb,rnn
elasticnet,1.0,0.000000e+00,0.000000e+00
lgb,0.0,1.000000e+00,4.444382e-87
rnn,0.0,4.444382e-87,1.000000e+00


In [24]:
# 查看彼此之間的中位數 # 結論elasticnet > rnn = lgb
print(statistics.median(lgb_test_abs))
print(statistics.median(rnn_test_abs))
print(statistics.median(elasticnet_test_abs))

0.2674252386020717
0.25213483577908136
0.4958753224136281


## 簡單看一下單純相減的結果

In [25]:
stkr(lgb_test, rnn_test, elasticnet_test)

KruskalResult(statistic=1914.6000242861808, pvalue=0.0)

In [26]:
data=pd.DataFrame({"rnn":rnn_test,"lgb":lgb_test,"elasticnet":elasticnet_test})
data = data.melt(var_name='groups', value_name='values')
sp.posthoc_dunn(data,"values","groups","bonferroni")

,elasticnet,lgb,rnn
elasticnet,1.000000e+00,5.290818e-294,0.000000
lgb,5.290818e-294,1.000000e+00,0.059656
rnn,0.000000e+00,5.965644e-02,1.000000


In [27]:
#結論: elasticnet > rnn > lgb(與50000筆結論相同)
print(statistics.median(lgb_test))
print(statistics.median(rnn_test))
print(statistics.median(elasticnet_test))

0.0173252898500244
0.014558251091027551
0.0904102143054639
